# PRÁCTICA 3: PROBLOG

Documentación interesante:

https://problog.readthedocs.io/en/latest/prolog.html

https://problog.readthedocs.io/en/latest/modeling_basic.html

In [197]:
!pip install problog

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [198]:
from problog.program import PrologString
from problog import get_evaluatable

## UTILITIES

These functions connect the structures of ProbLog to Python structures.

In [199]:

def make_queries(facts):
   return " ".join([' query(%s). '%fact for fact in facts])

def check_goal(result, facts):
  out = True
  for fact in facts:
     fact = fact.replace(" ","") #remove white spaces
     out = out and fact in [str(e) for e in result.keys() if result[e]>0]
  return out

def true_facts(result):
  return [str(e) for e in result.keys() if result[e]>0]

def add_action(plan, action):
  action = "do_" + action
  plan.append(action)
  return plan

def check_actions(result, actions):
  return [str(e) for e in result.keys() if result[e]>0 and str(e) in actions]


## LOGICAL PROBLEMS IN PROBLOG

The INITIAL STATE is now expressed as facts.

**Exercise**: Please, complete with your solution developed in the previous session.

In [200]:
initial ='''

 at(agent,room1).
 at(key1,room1).
 connected(door1,room1,room2).
 state(door1,closed).
 opens(key1,door1).
 holds(agent,none).
 free(key1).
 
'''

The AXIOMS express the pre-conditions and effects of actions. 

**Exercise**: Please, complete with your solution developed in the previous session. Read the document about this session to understand the difference between `grasp` and `do_grasp`.

In [201]:
axioms = '''

grasp(A, K) :-
    at(A, R),
    at(K, R),
    free(K).

do_grasp(A, K):-
    assertz(holds(A, K)),
    retract(holds(A, none)),
    retract(free(K)).

get-open(A, D) :-
    state(D, closed),
    opens(K, D),
    holds(A, K).

do_get-open(A, D):-
    assertz(state(D, open)),
    retract(state(D, closed)).

move(A, R1, R2) :-
      connected(D, R1, R2),
      state(D, open),
      R1 \= R2,
      at(Agent, R1).

do_move(A, R1, R2):-
    assertz(at(A, R2)),
    assertz(at(key1, R2)),
    retract(at(A, R1)),
    retract(at(key1, R1)).

query(use_module(library(assert))).


'''

## ASK & TELL

Now we implement the ASK & TELL mechanism so that the agent can choose and perform actions. See the document of this session for more details.

In [202]:
import random

**Exercise**: Complete your set of all actions in the next cell.

In [203]:
all_actions = ["grasp(agent,key1)", "get-open(agent,door1)", "move(agent,room1,room2)"]

The following functions implement the ASK&TELL mechanism.

In [204]:
def ASK_action(initial, axioms, plan, all_actions):

   KB = initial + axioms + make_queries(plan) + make_queries(all_actions)

   result = get_evaluatable().create_from(PrologString(KB)).evaluate()

   doable = check_actions(result, all_actions)

   return doable

def TELL_action(initial, goal, axioms, plan):
  
  KB = initial + axioms + make_queries(plan) + make_queries(goal)

  result = get_evaluatable().create_from(PrologString(KB)).evaluate()

  return check_goal(result, goal)

Creating a plan:

In [205]:
plan = []

acts = ASK_action(initial, axioms, plan, all_actions)

plan = add_action(plan, random.choice(acts))

plan

['do_grasp(agent,key1)']

Specifying a goal (it can include several facts):

In [206]:
goal = ["get-open(agent,door1)"]

Executing the plan and checking the goal:

In [207]:
TELL_action(initial, goal, axioms, plan)

True

## AGENT'S LOOP TO SOLVE A PROBLEM


**Exercise**: Complete and execute the next cell for your problem.

In [208]:
plan = []
path = []

for i in range(len(all_actions)):
  acts = ASK_action(initial, axioms, plan, all_actions)
  choice = random.choice(acts)
  plan = add_action(plan, choice)
  path.append(choice)

print(path)
print(plan)

['grasp(agent,key1)', 'get-open(agent,door1)', 'move(agent,room1,room2)']
['do_grasp(agent,key1)', 'do_get-open(agent,door1)', 'do_move(agent,room1,room2)']
